# **Practical Part**
### Mohammad Reza Nemati - 810100226
---
## **Generic Algorithm**
### Summery:
In this part the fractional knapsack problem is provided to be solved by generic algorithm.
<br>In contrast with classic binary knapsack, which each object could be selected or not (0/1), in fractional knapsack any portion *(or a limited portion, it has different variations)* of the object can be selected.
<br><br>In addition to the primary objective of the knapsack problem, the solution is also expected to satisfy a number of additional constraints to ensure its feasibility and effectiveness.

---


### Prerequisites:
We need to read and save the snacks data from csv file using NumPy library.
<br>It has a set of varied functions for random number generations which some of them are used in the project.

In [13]:
import numpy as np

data_array = np.genfromtxt('./data/snacks.csv', delimiter=',', skip_header=1, \
                           dtype=[('snack', 'U20'), ('weight', 'f8'), ('value', 'f8')])

Then we make two data classes:

`Const` class for storing hyperparameters of the generic algorithms:

- `max_generations`: The maximum number of generations that the genetic algorithm will run for.
- `population_size`: The number of chromosomes in the population.
- `crossover_probability`: The probability of performing a crossover operation on a pair of chromosomes.
- `mutation_probability`: The probability of performing a mutation operation on an chromosome.
- `elite_percentage`: The percentage of the population that will be preserved as-is (the "elites") in each generation.

<br>`Constraints` class for the provided constrainst in fraction knapsack problem:
- `max_weight`: The maximum weight that the knapsack can hold.
- `min_value`: The minimum value that should be picked.
- `min_type_number`: The minimum number of different types of objects that must be included in the knapsack.
- `max_type_number` : The maximum number of different types of objects that can be included in the knapsack.

In [14]:
from typing import NamedTuple

class Consts(NamedTuple):
    max_generations: int
    population_size: int
    crossover_probability: float
    mutation_probability: float
    elite_percentage: float

class Constraints(NamedTuple):
    max_weight: float
    min_value: float
    min_type_number: int
    max_type_number: int


Then we should instansiate the data classes.
<br>For our constants, these values are selected after various tests.
<a id='consts'></a>

In [24]:
consts = Consts(
    max_generations=700,
    population_size=500,
    crossover_probability=0.8,
    mutation_probability=0.3,
    elite_percentage=0.1
)

constraints = Constraints(
    max_weight=10,
    min_value=18, 
    min_type_number=2,
    max_type_number=4
)

<a id='basic'></a>
### Part 1: Defining Basic Concepts

The `Chromosome` class represents a potential solution to the fractional knapsack problem. Each chromosome is a sequence of genes, where each gene represents the fraction of a particular snack to include in the knapsack.

The class has the following attributes:

- `genes`: A numpy array of floats representing the fraction of each snack to include in the knapsack. A value of 0 means the snack is not included, while a value between 0 and 1 represents the fraction of the snack to include.
- `weight`: The total weight of the snacks included in the knapsack for this chromosome.
- `fitness`: The total value of the snacks included in the knapsack for this chromosome.

The class has the following methods:

- `__init__`: The constructor method. It initializes the genes with random values, and then calls the `calc_fitness` to store the fitness.
- `calc_fitness`: This method calculates the fitness of the chromosome. It sums up the weight and value of the snacks included in the knapsack, and sets the fitness to 0 if the weight exceeds the maximum allowed weight or the value is less than the minimum required value, or the number of different types of snacks is not within the specified range.
    - Note that the weight of the each object is calculated based on fraction of the snack to include which is saved in `genes` field.

In [16]:
class Chromosome:
    def __init__(self):
        self.genes = np.random.randint(0, 2, len(data_array)) * np.random.uniform(0, 1)
        self.weight = 0
        self.fitness = 0
        self.calc_fitness()
    
    def calc_fitness(self) -> None:
        self.weight = 0
        self.fitness = 0
        num_types = np.sum(self.genes != 0)

        for i in range(len(data_array)):
            if (self.genes[i] == 0):
                continue
            self.weight += self.genes[i] * data_array[i]['weight']
            self.fitness += self.genes[i] * data_array[i]['value']
        
        if self.weight <= constraints.max_weight and self.fitness >= constraints.min_value and \
            num_types in range(constraints.min_type_number, constraints.max_type_number+1):
            return
        
        self.fitness = 0

### GenericKnapsack class

The `GenericKnapsack` class is designed to solve the knapsack problem using a genetic algorithm. The class contains several methods that correspond to different steps of the genetic algorithm.

The class has the following attributes:

- `generations_max`: A list to store the best individual of each generation.
- `sorted_generations_max`: A sorted list of the best individuals from each generation.
- `population`: The current population of individuals.
- `elite_number`: The number of elite individuals to be carried the next generation.

The class has the following methods:

- `__init__`: Initializes the class attributes.
- `generate_initial_population`: Generates the initial population of individuals and sorts them based on their fitness.
- `select_elites`: Selects the elite individuals from the population.
- `random_point_crossover`: Performs a one-point crossover operation on two parent individuals to generate two offspring.
- `mutate`: Performs a mutation operation on an individual.
- `evolve`: Evolves the population by selecting elites, performing crossover and mutation operations, and generating a new population.
- `solve`: Solves the knapsack problem by evolving the population over a certain number of generations.
- `print_ans`: Prints the solution to the knapsack problem.
- `plot_ans`: Plots the fitness of the best individual in each generation.

In [17]:
class GenericKnapsack:
    def __init__(self):
        self.generations_max = []
        self.sorted_generations_max = []
        self.population = []
        self.elite_number = int(consts.elite_percentage * consts.population_size)

### Part 2: Generating initial population


The `generate_initial_population` method is responsible for creating the initial population of individuals (chromosomes) that the genetic algorithm will start with.


It creates the initial population using list comprehension to create a list of `Chromosome` instances. The number of instances is determined by `consts.population_size`, which represents the size of the population in the genetic algorithm.
Finally it sorts the population list based on the `fitness` of the chromosomes so we can pick the elites in the next step.

In [18]:
def generate_initial_population(self) -> None:
    self.population = [Chromosome() for _ in range(consts.population_size)]
    self.population.sort(key=lambda x: x.fitness, reverse=True)
    
GenericKnapsack.generate_initial_population = generate_initial_population

### Part 3: Implimenting fitness function

We have implemented `calc_fitness` method in `Chromosome` class.
<br>
[See more details here](#basic).

### Part 4: Elite-Surviving, Crossover, Mutation & Evolving

- `select_elites`: This method is part of the selection process in the genetic algorithm. It selects a subset of the population, specifically the top-performing individuals, to be preserved into the next generation. This is based on the concept of "survival of the fittest".

- `random_point_crossover`: This method implements the crossover operation, a fundamental genetic operator that combines the genetic information of two parent individuals to create new offspring. We have used one-point crossover, where a point on the parent chromosome strings is selected <b>randomly</b> and all data beyond that point in the two parent chromosomes is swapped to create new offspring.

- `mutate`: This method implements the mutation operation, another fundamental genetic operator that introduces small random change in the individuals' genetic composition. This is crucial for maintaining genetic diversity in the population and for introducing new genetic material. It should be noted that only one gene is picked to be mutated and will be mutetd by the `mutation_probability` which is a hyperparameter.

- `evolve`: This method is the core of the genetic algorithm in the `GenericKnapsack` class. It is responsible for evolving the current population to the next generation.
    - This method starts by selecting the elite individuals from the current population. It then enters a loop where it continually selects two parents at random and performs a crossover operation to produce offspring. These offspring are temporarily stored and then added to the new population.

    - The loop continues until the new population is filled up to the desired size, taking into account the number of elite individuals. After the new population is formed, a mutation operation is performed on each chromosome.

    - The `evolve` method returns the new population, which represents the next generation in the genetic algorithm. This new generation is expected to contain individuals that are, on average, more fit than those in the previous generation, thanks to the selection, crossover, and mutation operations.



In [19]:
def select_elites(self) -> list:
    return self.population[:self.elite_number]

def random_point_crossover(self, p1, p2) -> tuple:
    point = np.random.randint(1, len(p1.genes))
    child1 = Chromosome()
    child1.genes = np.concatenate((p1.genes[:point], p2.genes[point:]))
    child2 = Chromosome()
    child2.genes = np.concatenate((p2.genes[:point], p1.genes[point:]))

    return child1, child2

def mutate(self, chromosome) -> None:
    to_mut = np.random.randint(0, len(chromosome.genes))
    if np.random.uniform(0, 1) <= consts.mutation_probability:
        chromosome.genes[to_mut] = np.random.randint(0, 2) * np.random.uniform(0, 1)
    
    chromosome.calc_fitness()

def evolve(self, population) -> np.array:
    new_population = self.select_elites()
    temp_population = []
    while len(new_population) <= consts.population_size:
        parent1, parent2 = np.random.choice(population, 2, replace=False)
        if np.random.uniform(0, 1) < consts.crossover_probability:
            child1, child2 = self.random_point_crossover(parent1, parent2)
            temp_population += [child1, child2]

        new_population += temp_population[:2]
        temp_population = []

    for chromosome in new_population:
        self.mutate(chromosome)
    
    return new_population

GenericKnapsack.select_elites = select_elites
GenericKnapsack.random_point_crossover = random_point_crossover
GenericKnapsack.mutate = mutate
GenericKnapsack.evolve = evolve

### Part 5: Running the genetic algortihm

- `solve`: This method is the interface for running the algortihm. it loops to the number of `max_generations`, evolves the returned population and stores the best chromosome in that generation.

- `print_ans` & `plot_ans`: These are some helper methods for displaying outputs.

In [20]:
def solve(self) -> None:
    self.generation_max = []
    self.generate_initial_population()
    for _ in range(consts.max_generations):
        self.population = self.evolve(self.population)
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        self.generation_max.append(self.population[0])

    self.sorted_generation_max = sorted(self.generation_max, key=lambda x: x.fitness, reverse=True)

def print_ans(self) -> None:
    for i in range(len(self.sorted_generation_max[0].genes)):
        if (self.sorted_generation_max[0].genes[i] == 0):
            continue
        print(f"{data_array[i]['snack']}: {self.sorted_generation_max[0].genes[i] * data_array[i]['weight']}")
    print(f"Total Weight: {self.sorted_generation_max[0].weight} \
            \nTotal Value: {self.sorted_generation_max[0].fitness}")

def plot_ans(self) -> None:
    import matplotlib.pyplot as plt
    plt.scatter(range(len(self.generation_max)), [x.fitness for x in self.generation_max], \
                s=20, color='red', edgecolors='blue')
    plt.xlabel('generation')
    plt.ylabel('fitness')
    plt.show()

GenericKnapsack.solve = solve
GenericKnapsack.print_ans = print_ans
GenericKnapsack.plot_ans = plot_ans
                

### Part 6: Result evaluation:
We have tried different constraints and constants and the results show that in avarage, after generations the final tends to increase and get closer to the maximum. We also can see this flow in these diagrams.
<br>
Also when then crossover range and mutataion range are low, it seems that the overall answer does not change much with passing of generations and are mostly static. With these hyperparameters, if we have a high elite percentage, we can still reach good answers.
<br>However if crossover range and mutataion range are high, we can see that the answer tends to grow and mutate much faster.
<br><br><br>
[Click here to change the hyperparameters and constraints](#consts).
Or you can change them here.

In [39]:
consts = Consts(
    max_generations=500,
    population_size=200,
    crossover_probability=0.8,
    mutation_probability=0.3,
    elite_percentage=0.2
)


def run(knapsack):
    knapsack.solve()
    knapsack.print_ans()
    knapsack.plot_ans()

constraints = Constraints(
    max_weight=10,
    min_value=18, 
    min_type_number=2,
    max_type_number=4
)
if __name__ == "__main__":
    knapsack = GenericKnapsack()
    run(knapsack)

MazMaz: 0.0
Doogh-e-Abali: 0.0
Nani: 0.0
Jooj: 0.9101560175660325
Hot-Dog: 0.0
Chips: 0.0
Nooshaba: 0.0
Shokolat: 0.0
Chocoroll: 0.0
Cookies: 0.0
Abnabat: 0.0
Adams-Khersi: 0.0
Popcorn: 0.0
Pastil: 0.7439780943849617
Tordilla: 0.0
Masghati: 0.0
Ghottab: 0.0
Saghe-Talaei: 0.0
Choob-Shoor: 0.0
Total Weight: 8.603026406117113
Total Value: 18.860186924185218


In [38]:
consts = Consts(
    max_generations=500,
    population_size=200,
    crossover_probability=0.8,
    mutation_probability=0.3,
    elite_percentage=0.2
)

constraints = Constraints(
    max_weight=30,
    min_value=40.01, 
    min_type_number=8,
    max_type_number=10
)
knapsack = GenericKnapsack()
run(knapsack)

MazMaz: 0.0
Doogh-e-Abali: 0.0
Nani: 0.2490138398048246
Jooj: 0.940701014343966
Hot-Dog: 0.0
Chips: 0.0
Nooshaba: 0.0
Shokolat: 0.0
Chocoroll: 0.6297070123593433
Cookies: 0.0
Abnabat: 0.3801241444277158
Adams-Khersi: 0.0
Popcorn: 0.0
Pastil: 0.9095860288439412
Tordilla: 0.0
Masghati: 0.11516811981275521
Ghottab: 0.7372129817023145
Saghe-Talaei: 0.23139802376165153
Choob-Shoor: 0.0
Total Weight: 25.565507759743504
Total Value: 41.40818413939203


In [36]:
consts = Consts(
    max_generations=600,
    population_size=500,
    crossover_probability=0.8,
    mutation_probability=0.3,
    elite_percentage=0.2
)
constraints = Constraints(
    max_weight=55,
    min_value=65.01, 
    min_type_number=15,
    max_type_number=17
)

knapsack = GenericKnapsack()
run(knapsack)

MazMaz: 0.16951057217538468
Doogh-e-Abali: 0.008317047186853
Nani: 0.29922896858793624
Jooj: 0.9828632754424292
Hot-Dog: 0.0
Chips: 0.040850412273975234
Nooshaba: 0.0
Shokolat: 0.43196025457022
Chocoroll: 0.7176626662961152
Cookies: 0.1997371135386954
Abnabat: 0.42805525989929405
Adams-Khersi: 0.044275096622784504
Popcorn: 0.13035031323174562
Pastil: 0.9805117133872279
Tordilla: 0.1294377065635206
Masghati: 0.8852772666041405
Ghottab: 0.7750435367986946
Saghe-Talaei: 0.6081129096320798
Choob-Shoor: 0.38823640065848075
Total Weight: 50.79605636747865
Total Value: 68.33920478377657


In [37]:
consts = Consts(
    max_generations=1000,
    population_size=600,
    crossover_probability=0.8,
    mutation_probability=0.3,
    elite_percentage=0.2
)

constraints = Constraints(
    max_weight=80,
    min_value=87.01, 
    min_type_number=19,
    max_type_number=19
)
knapsack = GenericKnapsack()
run(knapsack)

MazMaz: 0.7759111457175396
Doogh-e-Abali: 0.022034877947924136
Nani: 0.38303429238235276
Jooj: 0.9579007623712258
Hot-Dog: 0.0727131727879835
Chips: 0.2661234935359794
Nooshaba: 0.22381594622254541
Shokolat: 0.6528050695432839
Chocoroll: 0.9959577050816957
Cookies: 0.33523218005847066
Abnabat: 0.6219703458476191
Adams-Khersi: 0.044743600463633015
Popcorn: 0.05475106474092972
Pastil: 0.8611313634748613
Tordilla: 0.5657422877169656
Masghati: 0.6170967428871332
Ghottab: 0.9898193066008026
Saghe-Talaei: 0.9281816929510762
Choob-Shoor: 0.39405301211516386
Total Weight: 75.26882697528936
Total Value: 92.21113723952391


---
### Questions

#### 1. *What are the implications of having a population size that is either too large or too small?*
  - A large population size can potentially expedite the discovery of the optimal solution due to increased diversity. However, it may also lead to higher resource consumption in terms of time and memory.
  - Conversely, a small population size may not provide sufficient diversity, which could result in the inability to find an optimal solution. But it could be more optimized it terms of time and memory.


#### 2. *What are the consequences if the population size expands with each generation?*
  - If the population size grows after each generation, it could enhance diversity, which might be beneficial. However, this would also escalate the consumption of resources (time and memory) with each step, which could be problematic.
  - Also maintaining a constant population size aids in the convergence of our chromosomes towards an optimal solution. Increasing the population size could potentially hinder this convergence process.


#### 3. *What is the impact of crossover and mutation? Can we use only one of them?*
  - Crossover and mutation are two fundamental genetic operations. Crossover enhances the chromosomes by merging beneficial traits from two parent chromosomes, while mutation directly alters a chromosome, aiding in escaping local extrema. For instance, observing the best fitness in each generation's population, we notice a convergence to a certain value, which is then disrupted and potentially increased by a mutation.
  - It might be possible to find a solution using only one of these operations, but it's likely to be inefficient. For example, relying solely on uniform crossover could lead to a solution, but it might require an excessive number of generations. Generally, using only mutation is unlikely to lead us to an optimal solution as it lacks the combinatorial benefit of crossover. However, in certain scenarios with specific constants, using only mutation could lead to a solution.

#### 4. *What strategies can we employ to speed up the algorithm?*
  - The efficiency of our algorithm can be improved by carefully selecting the appropriate values for all constants. Factors such as the number of generations, population size, and the probabilities of mutation and crossover should be chosen judiciously.

#### 5. *What strategies can we employ to address the issue of chromosomes becoming static after several generations?*
  - Mutation can be a solution to this problem as it introduces variability into the population, despite the crossover operation. 
  - Additionally, implementing a multi-start strategy can enhance the likelihood of discovering the global optimum as opposed to a local maximum.

#### 6. *What measures can we take to halt the algorithm if no solutions are found?*
  - One approach could be to set an upper limit for the number of generations, which would stop the algorithm once this limit is reached. Another strategy could be to compute the variances of the top-performing chromosomes over recent generations and compare this with a predefined threshold. The algorithm could then be terminated if the variance falls below this threshold.